In [1]:
import torch
from torchvision.models.detection import retinanet_resnet50_fpn
from torchvision.transforms import functional as F
from PIL import Image

import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [3]:
torch.cuda.is_available()

True

In [4]:
torch.device

torch.device

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
device

device(type='cuda')

In [3]:
# Import the 'multiprocessing' module to work with multi-processing features.
import multiprocessing

# Use 'multiprocessing.cpu_count()' to determine the number of available CPU cores.
cpu_count = multiprocessing.cpu_count()

In [4]:
cpu_count

8

In [6]:
import torchvision

In [7]:
torchvision.__version__

'0.16.2+cu118'

#### Load CIFAR10

In [25]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

cifar_train= datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

cifar_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

batch_size = 10

trainloader = torch.utils.data.DataLoader(cifar_train, batch_size=batch_size,
                                          shuffle=True, num_workers=4)

testloader = torch.utils.data.DataLoader(cifar_test, batch_size=batch_size,
                                         shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### Get Model

In [16]:
from torchvision.models import resnet50, ResNet50_Weights, resnet18

# Old weights with accuracy 76.130%
#resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

# New weights with accuracy 80.858%
#resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

# Best available weights (currently alias for IMAGENET1K_V2)
# Note that these weights may change across versions
resnet50(weights=ResNet50_Weights.DEFAULT)
resnet18(weights = ResNet

# Strings are also supported
#resnet50(weights="IMAGENET1K_V2")

# No weights - random initialization
#resnet50(weights=None)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [17]:
device = torch.device("cuda")

In [18]:
model = resnet50()

In [1]:
h : int = 1

In [2]:
h

1

#### Get Optimizer

In [19]:
import torch.optim as optim
from torch import nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

### Train

In [26]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    
    for i, data in enumerate(trainloader, 0):
        
        # get the inputs; data is a list of [inputs, labels]
        
        inputs, labels = data
        
        # zero the parameter gradients
        
        optimizer.zero_grad()

        # forward + backward + optimize
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        #print(data.shape)
        running_loss += loss.item()
        #if i % 2000 == 1999:    # print every 2000 mini-batches
        #    print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
        #    running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.775
[1,  4000] loss: 2.326


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001BA47721A80>
Traceback (most recent call last):
  File "C:\Users\akash\miniconda3\envs\ships\Lib\site-packages\torch\utils\data\dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "C:\Users\akash\miniconda3\envs\ships\Lib\site-packages\torch\utils\data\dataloader.py", line 1442, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "C:\Users\akash\miniconda3\envs\ships\Lib\multiprocessing\process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\akash\miniconda3\envs\ships\Lib\multiprocessing\popen_spawn_win32.py", line 109, in wait
    res = _winapi.WaitForSingleObject(int(self._handle), msecs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt: 
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001BA47721A80>
Traceback (most recent call 

KeyboardInterrupt: 

In [35]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [36]:
device

device(type='cuda')

In [21]:
len(trainloader)

10000

In [24]:
data

[tensor([[[[-0.0745,  0.1294,  0.2863,  ...,  0.5059,  0.4824,  0.5137],
           [-0.4431, -0.0745,  0.1059,  ...,  0.3333,  0.3412,  0.3255],
           [-0.3020, -0.0667, -0.0196,  ...,  0.1686,  0.1529,  0.1608],
           ...,
           [-0.1686, -0.2235, -0.2549,  ..., -0.3333, -0.3412, -0.3647],
           [-0.3882, -0.3961, -0.3804,  ..., -0.3098, -0.3176, -0.3412],
           [-0.3725, -0.3804, -0.3725,  ..., -0.3725, -0.3804, -0.3882]],
 
          [[-0.0118,  0.1922,  0.3569,  ...,  0.5686,  0.5451,  0.5686],
           [-0.3882, -0.0196,  0.1608,  ...,  0.3961,  0.4039,  0.3804],
           [-0.2471, -0.0039,  0.0353,  ...,  0.2314,  0.2235,  0.2235],
           ...,
           [-0.1922, -0.2392, -0.2706,  ..., -0.2784, -0.2941, -0.3176],
           [-0.3412, -0.3490, -0.3333,  ..., -0.2706, -0.2784, -0.3020],
           [-0.3333, -0.3412, -0.3412,  ..., -0.3647, -0.3725, -0.3647]],
 
          [[ 0.0745,  0.2784,  0.4353,  ...,  0.6627,  0.6392,  0.6706],
           [-